In [56]:
import sqlite3

##given the full name return symbol
def name2sym(string):
    conn = sqlite3.connect('share_data.db3')
    cursor = conn.cursor()
    string = string + '%'
    cursor.execute('select * from symbols where full_name like ?', (string,))
    result = cursor.fetchall()[0][0]
    cursor.close()
    conn.close()
    return result

##use symbol above to get share information at a given date
## symbol, date are both string
def query(symbol, date):
    conn = sqlite3.connect('share_data.db3')
    cursor = conn.cursor()
    cursor.execute("select * from '%s' where symbol=?"%date, (symbol,))
    a = cursor.fetchall()
    cursor.close()
    conn.close()
    openn = a[0][2]
    high = a[0][3]
    low = a[0][4]
    close = a[0][5]
    volume = a[0][6]
    adj_close = a[0][7]
    return openn, high, low, close, volume, adj_close

# buy = 'buy' or 'sell'
# type = 'cash' or 'spot'
def get_exchange_rate(money_name, buy, types):
    conn = sqlite3.connect('share_data.db3')
    cursor = conn.cursor()
    cursor.execute('select * from exchange_rate where money_name=?', (money_name,))
    result = cursor.fetchall()
    cursor.close()
    conn.close()
    if buy == 'buy':
        if types == 'cash':
            print('done')
            return result[0][3]
        elif types == 'spot':
            return result[0][4]
    elif buy == 'sell':
        if types == 'cash':
            return result[0][1]
        elif types == 'spot':
            return result[0][2]

?
done
30.792


In [56]:
import requests

#country12 are string
def exchange(country1, country2):
    r=requests.get('https://tw.rter.info/capi.php')
    currency=r.json()
    if country1 == 'USD':
        q = country1 + country2
        return float(currency[q]['Exrate'])
    elif country2 == 'USD':
        q = country2 + country1
        return 1 / float(currency[q]['Exrate'])
    else:
        q1 = 'USD' + country1
        q2 = 'USD' + country2
        return float(currency[q2]['Exrate']) / float(currency[q1]['Exrate'])

exchange('TWD', 'USD')

0.032671197072660745

In [9]:
from  exchange import exchange
from full_name2symbol import *

def RuleBasedNlg(action,slots):
    response = ''
    if action == 'exchange':
        rate = exchange(slots[0],slots[1])
        response = "The exchange rate from {0} to {1} is {2}".format( slots[0], slots[1], rate)

    elif action == 'get_exchange_rate':
        currency = slots[0]
        ex_type = slots[1]
        direction = slots[2]
        rate = get_exchange_rate(slots[0],slots[1],slots[2])
        response = "The {1} exchange rate for {0} NTD {2} {3} is {4} in BANK OF TAIWAN".format(ex_type+'ing', direction, 'from' if ex_type == 'buy' else 'for', currency, rate)

    elif action == 'query':
        symbol = name2sym(slot[0])
        date = slot[1]
        data = query(symbol,date)
        item = ['openn', 'high', 'low', 'close', 'volume']
        response = "Share price on date {0}  ".format(date)
        for i in range(len(item)):
            response += "{0} : {1}     ".format(item[i],data[i])

    return response 

slot = ['Apple','20170102']
RuleBasedNlg('query', slot)

'Share price on date 20170102  openn : 19.91     high : 20.00     low : 19.82     close : 19.98     volume : 1518900     '